## Ejemplo IoT Devices

#### Creamos la clase DeviceIoTData

In [0]:
%scala
case class DeviceIoTData(battery_level: Long, c02_level: Long, 
    cca2: String, cca3: String, cn: String, device_id: Long, 
    device_name: String, humidity: Long, ip: String, latitude: Double,
    lcd: String, longitude: Double, scale:String, temp: Long, timestamp: Long)

defined class DeviceIoTData

#### Creamos la clase DeviceTempByCountry

In [0]:
%scala
case class DeviceTempByCountry(temp: Long, device_name: String, device_id: Long, cca3: String)

defined class DeviceTempByCountry

#### Creamos el DataSet leyendo el fichero y usando la clase _DeviceIoTData_

In [0]:
%scala
val ds = spark.read.json("/databricks-datasets/learning-spark-v2/iot-devices/iot_devices.json").as[DeviceIoTData]

ds: org.apache.spark.sql.Dataset[DeviceIoTData] = [battery_level: bigint, c02_level: bigint ... 13 more fields]

#### Mostramos el esquema del DataSet

In [0]:
%scala
ds.printSchema

root
-- battery_level: long (nullable = true)
-- c02_level: long (nullable = true)
-- cca2: string (nullable = true)
-- cca3: string (nullable = true)
-- cn: string (nullable = true)
-- device_id: long (nullable = true)
-- device_name: string (nullable = true)
-- humidity: long (nullable = true)
-- ip: string (nullable = true)
-- latitude: double (nullable = true)
-- lcd: string (nullable = true)
-- longitude: double (nullable = true)
-- scale: string (nullable = true)
-- temp: long (nullable = true)
-- timestamp: long (nullable = true)

#### Mostramos los 5 primeros resultados

In [0]:
%scala
ds.show(5, false)

+-------------+---------+----+----+-------------+---------+---------------------+--------+-------------+--------+------+---------+-------+----+-------------+
battery_level|c02_level|cca2|cca3|cn |device_id|device_name |humidity|ip |latitude|lcd |longitude|scale |temp|timestamp |
+-------------+---------+----+----+-------------+---------+---------------------+--------+-------------+--------+------+---------+-------+----+-------------+
8 |868 |US |USA |United States|1 |meter-gauge-1xbYRYcj |51 |68.161.225.1 |38.0 |green |-97.0 |Celsius|34 |1458444054093|
7 |1473 |NO |NOR |Norway |2 |sensor-pad-2n2Pea |70 |213.161.254.1|62.47 |red |6.15 |Celsius|11 |1458444054119|
2 |1556 |IT |ITA |Italy |3 |device-mac-36TWSKiT |44 |88.36.5.1 |42.83 |red |12.83 |Celsius|19 |1458444054120|
6 |1080 |US |USA |United States|4 |sensor-pad-4mzWkz |32 |66.39.173.154|44.06 |yellow|-121.32 |Celsius|28 |1458444054121|
4 |931 |PH |PHL |Philippines |5 |therm-stick-5gimpUrBB|62 |203.82.41.9 |14.58 |green |120.97 |Celsius|25 |1458444054122|
+-------------+---------+----+----+-------------+---------+---------------------+--------+-------------+--------+------+---------+-------+----+-------------+
only showing top 5 rows

#### Creamos un nuevo DataSet con los datos de temperatura y humedad filtrando que la temperatura sea mayor a 30 y la humedad mayor a 70

In [0]:
%scala
val filterTempDs = ds.filter(data => {data.temp > 30 && data.humidity > 70})

filterTempDs: org.apache.spark.sql.Dataset[DeviceIoTData] = [battery_level: bigint, c02_level: bigint ... 13 more fields]

#### Mostramos los primeros 5 resultados

In [0]:
%scala
filterTempDs.show(5, false)

+-------------+---------+----+----+-------------+---------+----------------------+--------+---------------+--------+------+---------+-------+----+-------------+
battery_level|c02_level|cca2|cca3|cn |device_id|device_name |humidity|ip |latitude|lcd |longitude|scale |temp|timestamp |
+-------------+---------+----+----+-------------+---------+----------------------+--------+---------------+--------+------+---------+-------+----+-------------+
0 |1466 |US |USA |United States|17 |meter-gauge-17zb8Fghhl|98 |161.188.212.254|39.95 |red |-75.16 |Celsius|31 |1458444054129|
9 |986 |FR |FRA |France |48 |sensor-pad-48jt4eL |97 |90.37.208.1 |43.88 |green |4.9 |Celsius|31 |1458444054151|
8 |1436 |US |USA |United States|54 |sensor-pad-5410CWPrNb6|73 |204.15.64.249 |32.89 |red |-117.13 |Celsius|34 |1458444054155|
4 |1090 |US |USA |United States|63 |device-mac-63GL4xSaZbj|91 |66.198.198.1 |44.56 |yellow|-105.67 |Celsius|31 |1458444054162|
4 |1072 |PH |PHL |Philippines |81 |device-mac-81nsKomrRe |90 |222.127.71.1 |14.55 |yellow|121.04 |Celsius|31 |1458444054172|
+-------------+---------+----+----+-------------+---------+----------------------+--------+---------------+--------+------+---------+-------+----+-------------+
only showing top 5 rows

#### Creamos un nuevo DataSet usando la clase _DeviceTempByCountry_, primero filtramos que la temperatura sea mayor a 25, y luego con un _map_ iteraremos todos sus datos y le cambiaremos el nombre el nombre a las columnas

In [0]:
%scala
val dsTemp = ds.filter(data => {data.temp < 25}).map(data => (data.temp, data.device_name, data.device_id, data.cca3))
  .withColumnRenamed("_1", "temp")
  .withColumnRenamed("_2", "device_name")
  .withColumnRenamed("_3", "device_id")
  .withColumnRenamed("_4", "cca3").as[DeviceTempByCountry]

dsTemp: org.apache.spark.sql.Dataset[DeviceTempByCountry] = [temp: bigint, device_name: string ... 2 more fields]

#### Mostamos los 10 primeros resultados

In [0]:
%scala
dsTemp.show(10)

+----+--------------------+---------+----+
temp| device_name|device_id|cca3|
+----+--------------------+---------+----+
 11| sensor-pad-2n2Pea| 2| NOR|
 19| device-mac-36TWSKiT| 3| ITA|
 18|meter-gauge-7GeDoanM| 7| CHN|
 13| device-mac-9GcjZ2pw| 9| JPN|
 16|meter-gauge-11dlM...| 11| ITA|
 12|sensor-pad-12Y2kIm0o| 12| USA|
 13|meter-gauge-13Gro...| 13| IND|
 16|sensor-pad-14QL93...| 14| NOR|
 13| device-mac-15se6mZ| 15| USA|
 15|sensor-pad-16aXmI...| 16| USA|
+----+--------------------+---------+----+
only showing top 10 rows

#### Mostramos los primeros 5 resultados, filtrando que la temperatura sea mayor a 25, devolverá un objeto `Row` generico

In [0]:
%scala
ds.select($"temp", $"device_name", $"device_id", $"humidity", $"cca3", $"cn").where("temp > 25").show(5, false)

+----+---------------------+---------+--------+----+-------------+
temp|device_name |device_id|humidity|cca3|cn |
+----+---------------------+---------+--------+----+-------------+
34 |meter-gauge-1xbYRYcj |1 |51 |USA |United States|
28 |sensor-pad-4mzWkz |4 |32 |USA |United States|
27 |sensor-pad-6al7RTAobR|6 |51 |USA |United States|
27 |sensor-pad-8xUD6pzsQI|8 |35 |JPN |Japan |
26 |sensor-pad-10BsywSYUF|10 |56 |USA |United States|
+----+---------------------+---------+--------+----+-------------+
only showing top 5 rows

#### Mostraremos los 5 primeros resultados, en este caso se devolverá la clase _DeviceTempByCountry_

In [0]:
%scala
val dsTemp2 = ds.select($"temp", $"device_name", $"device_id", $"humidity", $"cca3", $"cn").where("temp > 25").as[DeviceTempByCountry]

dsTemp.show(5, false)

+----+---------------------+---------+----+
temp|device_name |device_id|cca3|
+----+---------------------+---------+----+
11 |sensor-pad-2n2Pea |2 |NOR |
19 |device-mac-36TWSKiT |3 |ITA |
18 |meter-gauge-7GeDoanM |7 |CHN |
13 |device-mac-9GcjZ2pw |9 |JPN |
16 |meter-gauge-11dlMTZty|11 |ITA |
+----+---------------------+---------+----+
only showing top 5 rows

dsTemp2: org.apache.spark.sql.Dataset[DeviceTempByCountry] = [temp: bigint, device_name: string ... 4 more fields]

#### Recogemos el primer resultado del DataSet _dsTemp_

In [0]:
%scala
val device = dsTemp.first()

device: DeviceTempByCountry = DeviceTempByCountry(11,sensor-pad-2n2Pea,2,NOR)

#### Mostramos los 5 primeros resultados, filtrando que el nivel de bateria sea menor a 8 y ordenado de menor a mayor según su nivel de CO2

In [0]:
%scala
ds.select($"battery_level", $"c02_level", $"device_name").where($"battery_level" < 8).sort($"c02_level").show(5, false)

+-------------+---------+---------------------------+
battery_level|c02_level|device_name |
+-------------+---------+---------------------------+
1 |800 |sensor-pad-186516qVnGVN7jK |
1 |800 |sensor-pad-190514UCQcNBzHH |
7 |800 |sensor-pad-1864985T2DlE87SJ|
4 |800 |meter-gauge-1855818YaYf |
3 |800 |sensor-pad-1886381NkVIS |
+-------------+---------+---------------------------+
only showing top 5 rows

#### Creamos un nuevo DataSet con los países que emiten más niveles de CO2

In [0]:
%scala
val newDs = ds.filter(data => {data.c02_level > 1300}).groupBy($"cn").avg().sort($"avg(c02_level)".desc)

newDs.show(10, false)

+------------------------------+------------------+--------------+--------------+-------------+-------------+------------------+---------+------------------+
cn |avg(battery_level)|avg(c02_level)|avg(device_id)|avg(humidity)|avg(latitude)|avg(longitude) |avg(temp)|avg(timestamp) |
+------------------------------+------------------+--------------+--------------+-------------+-------------+------------------+---------+------------------+
Solomon Islands |3.0 |1588.0 |187433.0 |40.0 |-9.43 |159.95 |21.0 |1.458444060894E12 |
Federated States of Micronesia|3.0 |1573.0 |78806.0 |55.0 |6.92 |158.25 |13.0 |1.45844405755E12 |
Rwanda |2.5 |1560.5 |102085.0 |44.0 |-2.0 |30.0 |21.5 |1.458444058393E12 |
British Indian Ocean Territory|7.0 |1560.0 |83930.0 |54.0 |-6.0 |71.5 |27.0 |1.458444057649E12 |
Aruba |4.0 |1559.0 |99899.0 |84.0 |12.51 |-70.0 |19.5 |1.458444057927E12 |
Isle of Man |5.0 |1548.0 |137958.5 |46.5 |54.23 |-4.57 |24.5 |1.4584440593755E12|
Gambia |3.0 |1544.5 |34748.0 |66.0 |13.47 |-16.57 |16.0 |1.458444056277E12 |
Lesotho |2.5 |1537.5 |74949.0 |72.0 |-29.41 |27.990000000000002|21.0 |1.458444057568E12 |
Cuba |5.2 |1534.8 |66819.2 |42.0 |21.444 |-79.336 |26.6 |1.4584440572212E12|
Gabon |8.0 |1523.0 |106953.0 |30.0 |0.38 |9.45 |28.0 |1.45844405847E12 |
+------------------------------+------------------+--------------+--------------+-------------+-------------+------------------+---------+------------------+
only showing top 10 rows

newDs: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [cn: string, avg(battery_level): double ... 7 more fields]

#### Mostramos los 10 primeros resultados, agrupados por países, con la media de temperatura y humedad

In [0]:
%scala
ds.filter(data => {data.temp > 25 && data.humidity > 75})
  .select("temp", "humidity", "cn").groupBy($"cn").avg().sort($"avg(temp)".desc, $"avg(humidity)".desc).as("avg_humidity").show(10, false)

+----------------------+---------+-------------+
cn |avg(temp)|avg(humidity)|
+----------------------+---------+-------------+
Monaco |34.0 |91.0 |
Anguilla |34.0 |83.0 |
British Virgin Islands|34.0 |81.0 |
Turkmenistan |34.0 |80.0 |
Suriname |34.0 |79.0 |
Gibraltar |34.0 |78.0 |
Liechtenstein |34.0 |76.0 |
Vanuatu |33.5 |84.0 |
Cameroon |33.0 |91.0 |
Fiji |33.0 |78.0 |
+----------------------+---------+-------------+
only showing top 10 rows

#### Mostramos los 10 primeros resultados de los valores mínimos y máximos de la temperatura, humedad, nivel de CO2

In [0]:
%scala
import org.apache.spark.sql.functions._

ds.select(min("temp"), max("temp"), min("humidity"), max("humidity"), min("c02_level"), max("c02_level"), min("battery_level"), max("battery_level")).show(10)

+---------+---------+-------------+-------------+--------------+--------------+------------------+------------------+
min(temp)|max(temp)|min(humidity)|max(humidity)|min(c02_level)|max(c02_level)|min(battery_level)|max(battery_level)|
+---------+---------+-------------+-------------+--------------+--------------+------------------+------------------+
 10| 34| 25| 99| 800| 1599| 0| 9|
+---------+---------+-------------+-------------+--------------+--------------+------------------+------------------+

import org.apache.spark.sql.functions._